# Homework 2
### DSCI 510 / USC / Hermjakob section / Spring 2023

Hello and welcome to HW2.

Guidelines:

Please write and submit the programs and data files below by the deadline: Friday, March 10, at 11:00pm Pacific time

You must complete the assignments individually. If you have trouble completing the assignment, please let one of the teaching assistants (TAs) know, during the lab or their office hours. They will help and guide you, but they will not write code for you and no one else should :) !!!

The submission will include several components
* HW2_[YOUR FIRSTNAME]_[YOUR LASTNAME].ipynb (for Q1, Q2, Q3)
* airports.tsv (result of Q2)
* airport-coordinates.tsv (result of Q3)
* ais.py (for Q4, Q5)

Please zip these files into "HW2_[YOUR FIRSTNAME]_[YOUR LASTNAME].zip" and upload it back to Blackboard for submission.
 
You may look up resources online like python docs and stackoverflow. You may look up topics, but not the questions themselves.

You can submit only one time. Your grade will be based on this submission.

# Q1 [15+3 points]

### Phone numbers (regular expressions)

Write a function that returns all valid North American phone numbers in a given string.<br>
You may assume that all phone numbers are given in the form (234) 567-8910.<br>
Neither the 3-digit area code nor the 7-digit phone number that follows may start with 0 or 1.<br>
Neither the 3-digit area code nor the 7-digit phone number that follows may have a 1 in both the second and third position, e.g. 911 is not a valid area code.

Bonus points for a short elegant solution using a regular expression.<br>
Regular expression reference: https://www.isi.edu/~ulf/teaching/python.html

In [3]:
from typing import List
import re
def find_valid_na_phone_numbers(s: str) -> List[str]:  # input: string; output: list of strings
    result = re.findall(r'(\([2-9](?!1{2})\d{2}\)\s[2-9](?!1{2})\d{2}\-\d{4})', s)
    return result

In [4]:
# Open test
print(find_valid_na_phone_numbers("She changed her number from (234) 567-8910 to (333) 333-3333 in 2021."))
# Should yield ["(234) 567-8910", "(333) 333-3333"]
print(find_valid_na_phone_numbers("He changed his number from (911) 567-8910 to (200) 100-1001 in 2019."))
# Should yield [] since both numbers are invalid.
#print(find_valid_na_phone_numbers("(213) 211-2134, (213) 123-2736"))

['(234) 567-8910', '(333) 333-3333']
[]


# Q2 [20 points]

### Overview Q2-Q5

The remaining questions (Q2-Q5) will guide you to build a simple __Airport Information System (AIS)__.
* Build a file airports.tsv with basic airport info, scraped from Wikipedia. (Q2)
* Build a file airport-coordinates.tsv with airport geo-coordinates, also scraped from Wikipedia. (Q3)
* Define a GeoCoordinates class and functions that convert between DMS and plain latitude/longitude format. (Q4)
* Define Airport and GeoAirport classes and write a Python program to answer the type of queries below. (Q5)
   * show airport info given an airport's IATA code
   * show airport info given part of an airport's name
   * show nearest airport(s) given geo-coordinates in either format
   
### Q2   
   
Wikipedia lists airports at https://en.wikipedia.org/wiki/List_of_airports_by_IATA_airport_code:_A
through https://en.wikipedia.org/wiki/List_of_airports_by_IATA_airport_code:_Z .

Write code that scrapes core airport info from those 26 Wikipedia pages: 
(1) IATA code, (2) name, (3) location, (4) URL,<br>
creating a file __airports.tsv__ with the 4 fields separated by tab (not comma). E.g. 
```
AAA	Anaa Airport	Anaa, Tuamotus, French Polynesia	https://en.wikipedia.org/wiki/Anaa_Airport
AAB	Arrabury Airport	Arrabury, Queensland, Australia	https://en.wikipedia.org/wiki/Arrabury_Airport
AAC	El Arish International Airport	El Arish, Egypt	https://en.wikipedia.org/wiki/El_Arish_International_Airport
...
```
We recommend that you first download the 26 Wikipedia webpages to files on your computer (caching),
and then extract the relevant data from those cached files. This avoids you having to download the same files from
the Web several times as you develop and test your extraction code. 
To further avoid bursts of requests, you might want to space out your requests
by "sleeping" (waiting) for a few seconds between requests.

Make sure to excludes footnotes such as "[1]" from IATA codes etc.<br>
IATA codes should be coomposed of three uppercase ASCII letters [A-Z].<br>
You will need the URLs to look up geo-coordinates for Californian airports in Q3.

In [ ]:
import time
import requests
from bs4 import BeautifulSoup

# download webpages
alphabet_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
for i in alphabet_list:
    time.sleep(5)   # wait for 5 seconds
    r = requests.get(f'https://en.wikipedia.org/wiki/List_of_airports_by_IATA_airport_code:_{i}')
    s = BeautifulSoup(r.content, 'html.parser')
    # find the table
    table = s.find('table', {'class': 'wikitable sortable'})
    # write in local files
    file_name = f'{i}.html'
    with open (file_name, 'w', encoding = 'utf-8') as f:
        f.write(str(table))

In [3]:
import requests
from bs4 import BeautifulSoup

alphabet_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
airport_1 = {}
# extract the relevant data from each file
for alphabet in alphabet_list:
    file_name = f'{alphabet}.html'
    with open(file_name, 'r') as f:
        # parse html 
        page_content = BeautifulSoup(f.read(), 'html.parser')
        rows = page_content.find_all('tr')[2:]
        
        for row in rows:
            td_tags = row.find_all("td")
            # skip the -AX- line (tr tag a tag)
            if not td_tags:
                continue
            # remove all sup tags 
            for td_tag in td_tags:
                while td_tag.sup:
                    td_tag.sup.decompose()
                while td_tag.span:
                    td_tag.span.decompose()
            
            # IATA Code
            IATA_code = td_tags[0].text.strip()
            
            # Airport name
            airport_name = td_tags[2].text.strip()
            
            # Airport name and URL
            a_tag = row.find('a')
            if not a_tag:
                airport_url = 'Not found'
            else:
                # revise the airport url
                airport_url = a_tag.get('href')
                if '/wiki/' in airport_url:
                    airport_url = airport_url.replace( '/wiki/', 'https://en.wikipedia.org/wiki/')
                else:
                    airport_url = 'Not found'
            # Airport location name
            location_name = td_tags[3].text.strip()
            
            # make a dictionary with attributes
            airport_1[IATA_code] = [airport_name, location_name, airport_url]
            
            # write in tsv file
with open ('airports.tsv', 'w', encoding = 'utf-8') as d:
    for x,y in airport_1.items():
        output = f"{x}\t{y[0]}\t{y[1]}\t{y[2]}\n"
        d.write(output)

# Q3. [20 points]

For the airports in _airports.tsv_ that you created in Q2, create a second file __airport-coordinates.tsv__ with the 
airports' geo-coordinates from the airports' Wikipedia pages.<br>
Limit this to airports in the US state of California (based on _airports.tsv_ info).
This will greatly limit the number of requests to the web and processing time on your computer.<br>
Store the data in tab-separated format.<br>
Fields: (1) IATA code (2) geo-coordinates. E.g.:
```
ACV     40°58′40″N 124°06′30″W
AHC     40°15′57″N 120°09′02″W
APC     38°12′47.50″N 122°16′50.50″W
...
```
As for Q2, we recommond caching the web pages on your computer and waiting a few seconds between requests.<br>
For almost all California airports, Wikipedia lists a normal airport Wikipedia page that contains coordinates.<br>
For the very small number of exceptions, where Wikipedia does not list a page with coordinates, 
you don't need to write anything to _airport-coordinates.tsv_, i.e.
no need to find the coordinates elsewhere on the Web.

In [1]:
import requests
from bs4 import BeautifulSoup

airport_2 = {}
# search those in CA
with open('airports.tsv', 'r') as f:
    for line in f:
        if line.strip(): # skip empty line
            object = line.strip('\n').split('\t')
        # access code and link
            # objct = code, airportname, location name, link
            if 'California, United States' in object[2]:
                airport_code = object[0]
                link = object[3]
                if link != 'Not found':
                    request_link = requests.get(link)
                    cor_page = BeautifulSoup(request_link.content, 'html.parser')
                    table_content = cor_page.find('span', {'class': 'plainlinks nourlexpansion'}) # find coordinates no matter where it is
                    if table_content.find('span', {'class': 'latitude'}) and table_content.find('span', {'class':'longitude'}):
                        latitude = table_content.find('span', {'class':'latitude'}).text
                        longtitude = table_content.find('span', {'class':'longitude'}).text
                        coordinates = f'{latitude} {longtitude}'
                    else:
                         coordinates = ''
                else:
                    coordinates = ''
                airport_2[airport_code] = coordinates

# write in tsv file
with open('airport-coordinates.tsv', 'w') as d:
    for x, y in airport_2.items():
        output = f'{x}\t{y}\n'
        d.write(output)

# Q4. [20+3 points]

Define a class __GeoCoordinates__ that stores geo-coordinates in both DMS (degree/minute/second) format 
and plain latitude/longitude format. Include functions that convert between the formats. A complete
GeoCoordinates instance will store the coordinates in both formats. For example, the geo-coordinates for USC are
* 34°01′18″N 118°17′07″W
* which is equivalent to a latitude of 34.02167 and a longitude of -118.28528

34°01′18″N is short for 34 degrees, 1 minute, 18 seconds north.<br>
There are 60 minutes in a degree, and 60 seconds in a minute.<br>
34°01′18″ = 34 + 1/60 + 18/3600 = 34.02167<br>
Southern latitudes and Western longitudes are expressed as negative values.<br>

An instance of a new _GeoCoordinates_ can be created by either providing a DMS (of type str) 
or latitude and and longitude (both float).

Variables for _GeoCoordinates_ should include dms (str), lat (float), long (float).<br>
Methods for _GeoCoordinates_ should include at least \_\_init\_\_(), \_\_str\_\_(), distance().
We provide the distance() method to you below.<br>
You probably want to write separate functions that convert from DMS to lat/long and vice versa.

In [4]:
from __future__ import annotations
from math import sin, cos, sqrt, atan2, radians
from typing import Optional, Tuple
import re

class GeoCoordinates:
    def __init__(self, lat: Optional[float] = None, long: Optional[float] = None, dms: Optional[str] = None):
        self.lat = lat
        self.long = long
        self.dms = dms
        if self.dms:
            self.convert_from_dms()
        elif self.lat and self.long:
            self.convert_from_latlong()


    def convert_from_dms(self) -> str:
        if self.dms:
            convert_type = self.dms.split(' ')
            la = convert_type[0]
            lo = convert_type[1]
            # convert latitudes
            if '″' in la:
                pattern = re.findall(r'^(?P<degree>[^°]+)\S(?P<minute>[^′]+)\S(?P<second>[^″]+)\S(?P<direction>[^\s]+)', la)
                for x, y, z, i in pattern:
                    degree = int(x)
                    minute = int(y)
                    second = float(z)
                    direction = i
                    lat = degree + (minute/60) + (second/3600)
                    if direction == 'S':
                        lat = -lat
                    else:
                        lat = lat
            else:
                pattern = re.findall(r'^(?P<degree>[^°]+)\S(?P<minute>[^′]+)\S(?P<direction>[^\s]+)',la)
                for x, y, i in pattern:
                    degree = int(x)
                    minute = int(y)
                    direction = i
                    lat = degree + (minute/60)
                    if direction == 'S':
                        lat = -lat
                    else:
                        lat = lat
            # convert longitudes
            if '″' in lo:
                pattern = re.findall(r'^(?P<degree>[^°]+)\S(?P<minute>[^′]+)\S(?P<second>[^″]+)\S(?P<direction>[^\s]+)', lo)
                for x, y, z, i in pattern:
                    degree = int(x)
                    minute = int(y)
                    second = float(z)
                    direction = i
                    long = degree + (minute/60) + (second/3600)
                    if direction == 'W':
                        long = -long
                    else:
                        long = long
            else:
                pattern = re.findall(r'^(?P<degree>[^°]+)\S(?P<minute>[^′]+)\S(?P<direction>[^\s]+)',lo)
                for x, y, i in pattern:
                    degree = int(x)
                    minute = int(y)
                    direction = i
                    long = degree + (minute/60)
                    if direction == 'W':
                        long = -long
                    else:
                        long = long
        self.lat = lat
        self.long = long
    
    def convert_from_latlong(self) -> str:
        if self.lat and self.long:
            # la
            if self.lat >= 0:
                la_degree = int(self.lat)
                la_min = (self.lat - la_degree)*60
                la_sec = (la_min - int(la_min))*60
                la = f'{la_degree}°{int(la_min)}′{int(la_sec)}″N'
            else:
                la_degree = int(abs(self.lat))
                la_min = (abs(self.lat) - la_degree)*60
                la_sec = (la_min - int(la_min))*60
                la = f'{la_degree}°{int(la_min)}′{int(la_sec)}″S'
            # lo
            if self.long >= 0:
                lo_degree = int(self.long)
                lo_min = (self.long - lo_degree)*60
                lo_sec = (lo_min - int(lo_min))*60
                lo = f'{lo_degree}°{int(lo_min)}′{int(lo_sec)}″E'
            else:
                lo_degree = int(abs(self.long))
                lo_min = (abs(self.long) - lo_degree)*60
                lo_sec = (lo_min - int(lo_min))*60
                lo = f'{lo_degree}°{int(lo_min)}′{int(lo_sec)}″W'
        self.dms = f'{la} {lo}'

    def __str__(self) -> str:
        output = f'{self.dms} (lat:{self.lat}, long:{self.long})'
        return output
   
    def distance(self, other: GeoCoordinates) -> float:
        earth_radius = 6373.0  # in kilometers
        lat1 = radians(self.lat)
        lon1 = radians(self.long)
        lat2 = radians(other.lat)
        lon2 = radians(other.long)
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = earth_radius * c
        return distance  # in kilometers

In [5]:
# open test
gc_la = GeoCoordinates(dms='34°03′N 118°15′W')
gc_sf = GeoCoordinates(lat=37.7775, long=-122.41639)
print('Coordinates of Los Angeles:', gc_la)
print('Coordinates of San Francisco:', gc_sf)
print(f'Straight-line distance from Los Angeles to San Francisco: {round(gc_la.distance(gc_sf))}km')

Coordinates of Los Angeles: 34°03′N 118°15′W (lat:34.05, long:-118.25)
Coordinates of San Francisco: 37°46′39″N 122°24′59″W (lat:37.7775, long:-122.41639)
Straight-line distance from Los Angeles to San Francisco: 559km


should print
```
Coordinates of Los Angeles: 34°03′N 118°15′W (lat: 34.05, long: -118.25)
Coordinates of San Francisco: 37°46′39″N 122°24′59″W (lat: 37.7775, long: -122.41639)
Straight-line distance from Los Angeles to San Francisco: 559km
```

# Q5. [25+3 points]

Define two classes _Airport_ and its subclass _GeoAirport_ for those airports that have an entry in _airport-coordinates.tsv_ (true for almost all California airports).<br>
_Airport_ instances will have variables for at least their IATA codes, their names, and their locations.<br>
_GeoAirport_ instances will also have variables for their geo-coordinates as well as a list of nearby airports, 
within a 100km range, technically a list of tuples (distance: float, nearby_airport: GeoAirport).<br>
Both should have reasonable \_\_str\_\_() methods. _GeoAirport_ should also have a method nearly_airports() that returns
a string with nearby airports in pretty format.<br>

The main program (including Q4 and Q5 code) should be a Python program __ais.py__ (Airport Information System)<br>
(You may develop your Q4 and Q5 code in Jupyter Notebook, if you wish, and then transfer to a regular Python program.)<br>
The main program will build a list or airports (Airport and GeoAirport) from the files airports.tsv and airport-coordinates.tsv that you built.<br>
The Python program should use the main programs arguments as captured in __sys.argv__<br>
It should process the following 6 formats:

* ais.py \<IATA-code\>
* ais.py \<DMS-lat\> \<DMS-long\>
* ais.py \<DMS-lat\> \<DMS-long\> \<number-of-nearest-airports\>
* ais.py \<latitude\> \<longitude\>
* ais.py \<latitude\> \<longitude\> \<number-of-nearest-airports\>
* ais.py \<airport-name-substring\> 

Apply the _substring_ format only if none of the above formats did not match.
The _substring_ can be anywhere in the airport's name, incl. in the middle.
    
## Examples

### Example 1 (IATA code of regular Airport)
Command: 
```
ais.py PEK
```
Print output:
```
PEK  Beijing Capital International Airport  (Beijing, China)
```

### Example 2 (IATA code of GeoAirport)
Command: 
```
ais.py LAX
```
Print output:
```
LAX  Los Angeles International Airport  (Los Angeles, California, United States; 33°56′33″N 118°24′29″W)
   Nearby:   7km HHR Hawthorne Municipal Airport (Jack Northrop Field)
   Nearby:   9km SMO Santa Monica Municipal Airport
   Nearby:  16km CPM Compton/Woodley Airport
   Nearby:  17km TOA Zamperini Field
   Nearby:  27km LGB Long Beach Airport
   ...
   Nearby:  92km RIR Flabob Airport
```

### Example 3 (substring of airport name)

Command: 
```
ais.py Oak
```
Print output:
```
OAK  Oakland International Airport  (Oakland, California, United States; 37°43′17″N 122°13′15″W)
ODC  Oakdale Airport (FAA: O27)  (Oakdale, California, United States; 37°45′23″N 120°48′01″W)
OKY  Oakey Army Aviation Centre  (Oakey, Queensland, Australia)
PTK  Oakland County International Airport  (Pontiac, Michigan, United States)
```
   
### Example 4 (USC coordinates, plain lat/long format, without number of nearest airports)

Command:
```
ais.py 34.02167 -118.28528
```
Print output:
```
 12km HHR  Hawthorne Municipal Airport (Jack Northrop Field)  (Hawthorne, California, United States; 33°55′22″N 118°20′07″W)
```

### Example 5 (USC coordinates, DMS format, with number of nearest airports)

Command:
```
ais.py 34°01′18″N 118°17′07″W 4
```
Print output:
```
 12km HHR  Hawthorne Municipal Airport (Jack Northrop Field)  (Hawthorne, California, United States; 33°55′22″N 118°20′07″W)
 14km LAX  Los Angeles International Airport  (Los Angeles, California, United States; 33°56′33″N 118°24′29″W)
 15km CPM  Compton/Woodley Airport  (Compton, California, United States; 33°53′24″N 118°14′37″W)
 15km SMO  Santa Monica Municipal Airport  (Santa Monica, California, United States; 34°00′57″N 118°27′05″W)
```

In [ ]:
class Airport:
    def __init__(self, code: str, airport_name: str, airport_location: str):
        self.code = code
        self.name = airport_name
        self.location = airport_location

    def __str__(self) -> str:
        output = f'{self.code}\t{self.name}\t({self.location})'
        return output

In [ ]:
class GeoAirport(Airport):
    def __init__(self,  code: str, airport_name: str, airport_location: str, geo_corrdinates: str, nearby: list[tuple] = []):
        super().__init__(code, airport_name, airport_location)
        self.geo = geo_corrdinates
        # within a 100km range, technically a list of tuples (distance: float, nearby_airport: GeoAirport)
        self.nearby = nearby
        
    def __str__(self):
        if self.geo:
            return f'{self.code}\t{self.name}\t({self.location};{self.geo})'
        else:
            return f'{self.code}\t{self.name}\t({self.location})'
    
    def nearly_airports(self):
        #returns a string with nearby airports in pretty format
        nearby_str = ''
        for near in self.nearby:
            dist = near[0]
            code = near[1].code
            name = near[1].name
            nearby_str += f"   Nearby:\t{dist:>2}km\t{code}\t{name}\n"
        return nearby_str

In [ ]:
# Airport instance
airport_instance_dict = {}
with open('airports.tsv', 'r') as f:
    for line in f:
        if line.strip():
            object = line.split('\t')
            code = object[0]
            airport_name = object[1]
            airport_location = object[2]
            #airport_instance_list.append(Airport(code, airport_name, airport_location))
            airport_instance_dict[code] = Airport(code, airport_name, airport_location)

In [ ]:
# Geoairport instance
airport_coordinate = {}
with open('airport-coordinates.tsv', 'r') as d:
    for x in d:
        if x.strip():
            object = x.split('\t')
            airport_coordinate[object[0]] = object[1].strip()

# nearby_airports: within a 100km range, technically a list of tuples (distance: float, nearby_airport: GeoAirport)
airports_GeoAirport = {}  # airports {airport: its GeoAirport}

# Build GeoAirport instances for all airports
for c, coord in airport_coordinate.items():
    code = airport_instance_dict[c].code
    name = airport_instance_dict[c].name
    airport_location = airport_instance_dict[c].location
    airports_GeoAirport[c] = GeoAirport(code, name, airport_location, geo_corrdinates = coord, nearby =[]) 
 
# Update nearby_airports for all airports (GeoAirport instance)  
for x, y in airport_coordinate.items():
    for a, b in airport_coordinate.items():
        # Skip comparing two identical airport
        if x == a:
            continue
        
        if y and b:  # Some coordinates are missing
            distance_km = round(GeoCoordinates(dms=y).distance(GeoCoordinates(dms=b)))
            if distance_km <= 100:
                # Append nearby airport to nearby_airport list of the current instance
                # print(f'from {x} to {a}: {distance_km} km')
                airports_GeoAirport[x].nearby.append((distance_km, airports_GeoAirport[a]))

    airports_GeoAirport[x].nearby.sort(key = lambda x: x[0])

In [ ]:
# example 1, 2
arg = "PEK"
arg = "ACV"
if arg in airports_GeoAirport:
    print(airports_GeoAirport[arg])
    print(airports_GeoAirport[arg].nearly_airports())
elif arg in airport_instance_dict:
    print(airport_instance_dict[arg])

In [ ]:
# example 3
arg = "Oak"
for airport in airport_instance_dict:
    if arg in airport_instance_dict[airport].name:
        print(airport_instance_dict[airport])

In [ ]:
# example 4
arg1, arg2 = "34.02167", "-118.28528"
# example 5
arg1, arg2 = "34°01′18″N", "118°17′07″W"
nums = 1

lat, long, dms = None, None, None
try:
    lat = float(arg1)
    long = float(arg2)
except:
    dms = f'{arg1} {arg2}'
    nums = 4

airports = []  # [(distance, GeoAirport)]
for c in airports_GeoAirport:
    current_airport = airports_GeoAirport[c]
    # print(current_airport)
    if current_airport.geo:
        dist = round(GeoCoordinates(dms=current_airport.geo).distance(GeoCoordinates(lat=lat, long=long, dms=dms)))
        airports.append((dist, current_airport))
        print(current_airport, dist)

airports.sort(key = lambda x: x[0]) 
print(airports[0])
for i in range(nums):
    print(f"{airports[i][0]}km\t{airports[i][1]}")  